<a href="https://colab.research.google.com/github/sofieneJ/classification_CES_project_sources/blob/master/keras_bert_fine_tune_consumer_complaints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
This work is based on Jacob Zweig's work https://github.com/strongio/keras-bert

##Init

In [0]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 5.8MB/s 


In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K
import keras
import gensim
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



### mounting to google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"


#@markdown 1- complaints_data_dir is the main dataset dir.
#@markdown under the complaints_data_dir there should be the raw csv file with two columns ['Product', 'Consumer complaint narrative']
complaints_data_dir = root_dir + 'my_colab_storage/consumer_complaints_data/'

#@markdown 2- The dir to train/test split csv files. This director should be under complaints_data_dir
evaluation_data_dir = complaints_data_dir+'train_test_data/'
tf.gfile.MakeDirs(evaluation_data_dir)

#@markdown 3- bert_path is the path to the BERT pretrained model dir. It can be replaced by the tensorflow-hub url: https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1
bert_path = root_dir +'my_colab_storage/pre-trained-models/tf_hub_bert_uncased_L-12_H-768_A-12/'
#@markdown 4- trained_model_path is the path to the trained model output
trained_model_path = root_dir+'my_colab_storage/bert_fine_tuning/keras/classification_model/KerasCustomerComplaintsModel.h5'



#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(trained_model_path)
  except:
    # Doesn't matter if the directory didn't exist
    pass
#tf.gfile.MakeDirs(trained_model_path)
print('***** Model output path: {} *****'.format(trained_model_path))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
***** Model output path: /content/gdrive/My Drive/my_colab_storage/bert_fine_tuning/keras/classification_model/KerasClassificationModel.h5 *****


In [0]:
max_seq_length = 256
min_seq_length = 10


## data

In [0]:
LABEL_COLUMN = 'Product'
DATA_COLUMN = 'Consumer complaint narrative'


def filter_subsample_split_data():
  # label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
  # label_list = []
  raw_data_file = complaints_data_dir +'Consumer_Complaints_light.csv'
  complains_df = pd.read_csv(filepath_or_buffer=raw_data_file, sep=',')
  my_categories = ['Debt collection', 'Mortgage', 'Credit reporting', 'Student loan', 'Credit card', 'Bank account or service', 
  'Checking or savings account', 'Consumer Loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service']

  my_categories = ['Debt collection', 'Mortgage', 'Credit reporting', 'Student loan', 'Credit card']

  complains_df = complains_df.loc[complains_df[LABEL_COLUMN].isin(my_categories)].reset_index(drop=True)
  print ('size after category filter ', complains_df.shape)
  # print (complains_df.head())
  count_cols = [LABEL_COLUMN, 'Count']
  complains_by_group = complains_df.groupby(by=LABEL_COLUMN,axis=0).count().reset_index().rename(columns = {DATA_COLUMN:'Count'})
  # print (type(complains_by_group))
  print (complains_by_group.sort_values(by="Count", ascending =False))


  ########################### Sub-sample the dataset #########################################
  index_reduced = np.arange(0,complains_df.shape[0])
  np.random.shuffle(index_reduced)
  kept_data_ratio = 0.1
  index_reduced = index_reduced[0:int(complains_df.shape[0]*kept_data_ratio)]
  complains_df = complains_df.loc[index_reduced]

  print ('complaints distribution after subsampling')
  count_cols = [LABEL_COLUMN, 'Count']
  complains_by_group = complains_df.groupby(by=LABEL_COLUMN,axis=0).count().reset_index().rename(columns = {DATA_COLUMN:'Count'})
  # print (type(complains_by_group))
  print (complains_by_group.sort_values(by="Count", ascending =False))
  print ('kept categories are:', np.sort(complains_df[LABEL_COLUMN].unique()))

  ########################### test train split the dataset #########################################
  cats = complains_df[LABEL_COLUMN]
  corpus = complains_df[DATA_COLUMN]
  corpus_train, corpus_test, cat_train, cat_test = train_test_split(corpus, cats, test_size = 0.3, random_state=100)
  
  ########################### Dump train test datasets #######################################
  train_test_data_dir = complaints_data_dir +'train_test_data/'
  corpus_train.to_csv(path_or_buf= train_test_data_dir+'train_corpus.csv', index = False, header = True)
  corpus_test.to_csv(path_or_buf= train_test_data_dir+'test_corpus.csv', index = False, header = True)
  cat_train.to_csv(path_or_buf= train_test_data_dir+'train_cat.csv', index = False, header = True)
  cat_test.to_csv(path_or_buf= train_test_data_dir+'test_cat.csv', index = False, header = True)
  print (f'train subset size: {corpus_train.shape}')
  print (f'test subset size: {corpus_test.shape}')
  
filter_subsample_split_data()

size after category filter  (212171, 2)
            Product  Count
2   Debt collection  86850
3          Mortgage  53056
1  Credit reporting  31588
4      Student loan  21839
0       Credit card  18838
complaints distribution after subsampling
            Product  Count
2   Debt collection   8595
3          Mortgage   5265
1  Credit reporting   3219
4      Student loan   2218
0       Credit card   1920
kept categories are: ['Credit card' 'Credit reporting' 'Debt collection' 'Mortgage'
 'Student loan']
train subset size: (14851,)
test subset size: (6366,)


In [0]:
def preprocess_data():
  
  REFUSED_DOC_TAG = '<<<refused_document>>>'
  def simple_preprocess_func (doc):
    seq = gensim.utils.simple_preprocess(gensim.parsing.remove_stopwords(doc), min_len=2, max_len=25)
    if len(seq)<min_seq_length or len(seq)>max_seq_length:
      return REFUSED_DOC_TAG
    else:
      return ' '.join(seq)
    
  
  train_test_data_dir = complaints_data_dir +'train_test_data/'
  subsets = ('train','test')
  dfs = []
  for subset in subsets:
    ############################# corpus preprocessing ######################################
    corpus_path = train_test_data_dir+f'{subset}_corpus.csv'
    corpus_ser = pd.read_csv(filepath_or_buffer=corpus_path, sep=',')[DATA_COLUMN]
    print (f'{subset} corpus size before preprocessing is {corpus_ser.shape}')
    corpus_ser = corpus_ser.apply(lambda txt : simple_preprocess_func(txt))

    valid_index = (corpus_ser != REFUSED_DOC_TAG).tolist()
    corpus_ser = corpus_ser[valid_index].reset_index(drop=True)
    print (f'{subset} corpus size after preprocessing is {corpus_ser.shape}')
    
    ###################################### Some statictics #####################################
    doc_lens = np.array([len(doc.split()) for doc in corpus_ser])
    print (f'mean length is {np.mean(doc_lens)}')
    print (f'max length is {np.max(doc_lens)}')

    
    ###################################### Named category conversion ##########################
    cat_path = train_test_data_dir+ f'{subset}_cat.csv'
    cats_ser = pd.read_csv(filepath_or_buffer=cat_path, sep=',')[LABEL_COLUMN]
    class_dico = {i:classe for i,classe in enumerate(np.sort(cats_ser.unique()))}
    print ('classes', class_dico)
    inv_class_dico = {item[1]:item[0] for item in class_dico.items()}
    cats_ser = cats_ser.apply(lambda x : inv_class_dico[x])
    cats_ser = cats_ser[valid_index].reset_index(drop=True)
    #preprocessed_cat_path = f'data\\{subset}_cat_reduced.csv'
    #cats_ser.to_csv(path_or_buf=preprocessed_cat_path, index=False)

    assert(cats_ser.shape[0]==corpus_ser.shape[0])
    
    ###################################### returning DataFrames ##########################
    data_dic = {DATA_COLUMN: corpus_ser, LABEL_COLUMN:cats_ser}
    subset_df = pd.DataFrame(data_dic)
    dfs.append(subset_df)
  
  label_list = [i for i in np.sort(dfs[0][LABEL_COLUMN].unique())]
  
  return dfs[0], dfs[1], label_list

train_df, test_df, label_list = preprocess_data()
print ('the label list is:', label_list)



train corpus size before preprocessing is (14851,)
train corpus size after preprocessing is (13386,)
mean length is 79.45472882115644
max length is 256
classes {0: 'Credit card', 1: 'Credit reporting', 2: 'Debt collection', 3: 'Mortgage', 4: 'Student loan'}
test corpus size before preprocessing is (6366,)
test corpus size after preprocessing is (5715,)
mean length is 78.87419072615923
max length is 255
classes {0: 'Credit card', 1: 'Credit reporting', 2: 'Debt collection', 3: 'Mortgage', 4: 'Student loan'}
the label list is: [0, 1, 2, 3, 4]


In [0]:

# Create datasets (Only take up to max_seq_length words for memory)
train_text = train_df[DATA_COLUMN].tolist()
train_text = np.array(train_text, dtype=str)[:, np.newaxis]
train_label = train_df[LABEL_COLUMN].tolist()
print (f' train text shape {train_text.shape}')


test_text = test_df[DATA_COLUMN].tolist()
test_text = np.array(test_text, dtype=str)[:, np.newaxis]
test_label = test_df[LABEL_COLUMN].tolist()
print (f' test text shape {test_text.shape}')

 train text shape (13386, 1)
 test text shape (5715, 1)
 train text shape (13386, 1)
 test text shape (5715, 1)


##BERT formating and tokenization

In [0]:
# Initialize session
sess = tf.Session()

class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
test_examples = convert_text_to_examples(test_text, test_label)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore



## model

In [0]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="first",
        bert_path=bert_path,
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_path = bert_path
        if self.pooling not in ["first", "mean"]:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_path, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                f"Undefined pooling type (must be either first or mean, but is {self.pooling}"
            )

        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError(f"Undefined pooling type (must be either first or mean, but is {self.pooling}")

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [0]:
# Build model
def build_model(max_seq_length): 
    in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=1, pooling="mean")(bert_inputs) #pooling="first"
    dense = tf.keras.layers.Dense(units=512, activation='relu')(bert_output) #units=256
    #dense2 = tf.keras.layers.Dense(units=256, activation='relu')(dense1) #units=256
    pred = tf.keras.layers.Dense(units=len(label_list), activation='softmax')(dense) #sigmoid
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #binary_crossentropy
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [0]:

model = build_model(max_seq_length)

# Instantiate variables
initialize_vars(sess)

categorical_train_labels = keras.utils.to_categorical(train_labels, num_classes=len(label_list))
categorical_test_labels = keras.utils.to_categorical(test_labels, num_classes=len(label_list))

current_time = datetime.now()
model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    categorical_train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], categorical_test_labels),
    epochs=2,
    batch_size=32
)
print("Training and evaluation took time ", datetime.now() - current_time)





Using TensorFlow backend.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 256)]        0                                            
_____________________________________________________________________________________

KeyboardInterrupt: ignored

In [0]:

model.save(trained_model_path)

## Predictions performance

In [0]:
model = build_model(max_seq_length)
initialize_vars(sess)
model.load_weights(trained_model_path)

post_save_preds = model.predict([test_input_ids, 
                                test_input_masks, 
                                test_segment_ids]
                            ) # predictions after we clear and reload model

# all(pre_save_preds == post_save_preds) # Are they the same?
print (post_save_preds.shape)
predicted_classes = np.argmax(post_save_preds, axis=1)


print (classification_report(test_labels,predicted_classes)) #,target_names = my_cats

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_layer_2 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
   

## prediction on test corpus

In [0]:
def predict_single_sample_with_finetuned_model(model, tokenizer, sample):
    seq = gensim.utils.simple_preprocess(gensim.parsing.remove_stopwords(sample), min_len=2, max_len=25)
    if (len(seq) > max_seq_length):
        raise NameError(
                f"sequence is too long: the obtained size after preprocessing is {len(seq)}. Maximum size is {max_seq_length}"
            )
    if (len(seq)<min_seq_length):
        raise NameError(
                f"sequence is too short: the obtained size after preprocessing is {len(seq)}. Minimum size is 10"
            )
    
    bert_example = convert_text_to_examples(texts = [seq], labels=[-1])
    # bert_sample = InputExample(guid=None, text_a=" ".join(seq), text_b=None, label=-1)
    # Instantiate tokenizer
    

    input_id, input_mask, segment_id, _ = convert_single_example(
        tokenizer, bert_example[0], max_seq_length
    )

    #model = build_model(max_seq_length)
    #initialize_vars(sess)
    #model.load_weights(trained_model_path)

    post_save_preds = model.predict([[input_id], 
                                    [input_mask], 
                                    [segment_id]]
                                ) # predictions after we clear and reload model

    # all(pre_save_preds == post_save_preds) # Are they the same?
    
    return post_save_preds[0]
  
  

In [0]:

dico= {0: 'Credit card', 1: 'Credit reporting', 2: 'Debt collection', 3: 'Mortgage', 4: 'Student loan'}
inv_class_dico = {item[1]:item[0] for item in dico.items()}

              
evaluation_data_dir = complaints_data_dir+'test_corpus/'
corpus_path = evaluation_data_dir+'test_corpus.csv'
label_path = evaluation_data_dir+'test_cat.csv'

example_ser = pd.read_csv(filepath_or_buffer=corpus_path, sep=',')[DATA_COLUMN]
cat_ser = pd.read_csv(filepath_or_buffer=label_path, sep=',')[LABEL_COLUMN]

evaluation_df = pd.DataFrame({DATA_COLUMN:example_ser, LABEL_COLUMN:cat_ser})
#print (evaluation_df.head(2))


###############loading model and tokenizer######################""
#tokenizer = create_tokenizer_from_hub_module()
#model = build_model(max_seq_length)
#initialize_vars(sess)
#model.load_weights(trained_model_path)


#############evaluation loop#############################
for index, row in evaluation_df.iterrows():
  example_text  = row[DATA_COLUMN]
  example_cat  = row[LABEL_COLUMN]

  
  try:
    liste_prob = predict_single_sample_with_finetuned_model (model, tokenizer, example_text)
    #if (np.argmax(liste_prob)!=inv_class_dico[example_cat]):
    print ('** Email text:', example_text)
    print ('** Real category:', example_cat)
    print ('** Predicted probabilities: ', dict(zip(inv_class_dico,liste_prob)))
    print ('*********************************************NEXT EXAMPLE********************************')
      
  except NameError:
    print ('preprocessing error')
  
  
  if index==20:
    break




** Email text: I was recently considered late on a payment ( payment was made just after XXXX on the due date ). The payment was made on the due date and the cut off is arbitrary for an online payment. I make all my payments online and Discover is the only company that imposes this. The second complaint is the amount of the late charge being excessive. The balance is {$1000.00} and my average monthly payment has been {$36.00}. The late charge alone was {$35.00} and doubled my most recent payment. Last, the 18 % interest rate is outrageous. I have good credit and my credit score is over XXXX. I can get a lower rate if I apply for a new card. I do n't want another card with Discover. I stopped using it when they changed a due without notification causing a late payment. They also sent the account to a collection agency even though it was not delinquent and never had been. It took months to be resolved. So NO WAY do I want to do business with Discover. 

What I want is the late charge red